## 1. Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
import plotly.express as px
import lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostRegressor, CatBoostClassifier

In [ ]:
# 페어쌍으로 묶인 아이템 시각화 비교 함수
def visualize_(df, pairs, ref_item, target_col=None):
    if target_col is None:
        target_col = np.append(["ym",ref_item],pairs[pairs["leading_item_id"] == ref_item]["following_item_id"].values)
    uss_df = df[target_col].copy()
    uss_long = uss_df.melt(id_vars="ym", var_name="item", value_name="value")
    bench = (
        uss_long.loc[uss_long["item"] == ref_item, ["ym", "value"]]
                .rename(columns={"value": "bench_value"})
    )
    dfc = uss_long.merge(bench, on="ym", how="left")
    plot_df = pd.concat(
        [
            dfc.assign(series="self",  val=dfc["value"]),
            dfc.assign(series=f"benchmark: {ref_item}", val=dfc["bench_value"])
        ],
        ignore_index=True
    )
    fig = px.line(
        plot_df,
        x="ym",
        y="val",
        color="series",
        facet_col="item",
        facet_col_wrap=6,
        facet_row_spacing=0.02,
        color_discrete_map={
            "self": "#2962FF",                 # 각 아이템(자기 자신)
            f"benchmark: {ref_item}": "#9E9E9E"  # 기준 라인(회색)
        }
    )

    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]),)
    fig.update_traces(opacity=0.95)
    # fig.update_layout(width=1800, height=1400, showlegend=True)
    fig.update_layout(showlegend=True,)

    fig.show()
    
# 단순 페어쌍 갯수 변화 함수
def count_p(pairs1,pairs2):
    count = {}
    plus = 0
    neg = 0
    pairs = np.union1d(pairs1["leading_item_id"].unique(), pairs2["leading_item_id"].unique())
    for i in pairs:
        target_col = np.append(["ym",i],pairs1[pairs1["leading_item_id"] == i]["following_item_id"].values)
        target_col_sub = np.append(["ym",i],pairs2[pairs2["leading_item_id"] == i]["following_item_id"].values)
        temp_inter = np.intersect1d(target_col,target_col_sub,)
        temp_tar = np.setdiff1d(target_col,target_col_sub,)
        temp_tar = np.append(["ym",i],temp_tar)
        temp_tar_sub = np.setdiff1d(target_col_sub,target_col,)
        temp_tar_sub = np.append(["ym",i],temp_tar_sub)
        count[i] = [len(target_col)-2,len(target_col_sub)-2, len(temp_inter)-2, len(temp_tar)-2,len(temp_tar_sub)-2]
        plus += len(temp_tar)-2
        neg += len(temp_tar_sub)-2
    return count, plus, neg, plus - neg

## 2. 데이터 전처리

In [2]:
train = pd.read_csv('./train.csv')

# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)
monthly["ym"] = monthly["ym"].dt.strftime("%Y-%m")

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

pivot.head()

ym,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,...,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,14276.0,52347.0,53549.0,0.0,26997.0,84489.0,0.0,0.0,0.0,0.0,...,428725.0,144248.0,26507.0,25691.0,25805.0,0.0,38441.0,0.0,441275.0,533478.0
AHMDUILJ,242705.0,120847.0,197317.0,126142.0,71730.0,149138.0,186617.0,169995.0,140547.0,89292.0,...,123085.0,143451.0,78649.0,125098.0,80404.0,157401.0,115509.0,127473.0,89479.0,101317.0
ANWUJOKX,0.0,0.0,0.0,63580.0,81670.0,26424.0,8470.0,0.0,0.0,80475.0,...,0.0,0.0,0.0,27980.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,383999.0,512813.0,217064.0,470398.0,539873.0,582317.0,759980.0,216019.0,537693.0,205326.0,...,683581.0,2147.0,0.0,25013.0,77.0,20741.0,2403.0,3543.0,32430.0,40608.0
ATLDMDBO,143097177.0,103568323.0,118403737.0,121873741.0,115024617.0,65716075.0,146216818.0,97552978.0,72341427.0,87454167.0,...,60276050.0,30160198.0,42613728.0,64451013.0,38667429.0,29354408.0,42450439.0,37136720.0,32181798.0,57090235.0


In [3]:
# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly_w = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly_w["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly_w["month"].astype(str).str.zfill(2)
)
monthly_w["ym"] = monthly_w["ym"].dt.strftime("%Y-%m")

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot_w = (
    monthly_w
    .pivot(index="item_id", columns="ym", values="weight")
    .fillna(0.0)
)

pivot_w.head()

ym,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,...,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,17625.0,67983.0,69544.0,0.0,34173.0,103666.0,0.0,0.0,0.0,0.0,...,786651.0,249144.0,33133.0,32937.0,33083.0,0.0,49050.0,0.0,865246.0,1046036.0
AHMDUILJ,100990.0,43444.0,64113.0,42637.0,21468.0,59424.0,61587.0,63625.0,61245.0,20382.0,...,42986.0,43763.0,24379.0,62351.0,23521.0,43332.0,44913.0,44035.0,25574.0,34463.0
ANWUJOKX,0.0,0.0,0.0,89967.0,118992.0,41649.0,13888.0,0.0,0.0,119940.0,...,0.0,0.0,0.0,37211.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,50193.0,81429.0,43310.0,62505.0,84680.0,37425.0,114600.0,39305.0,104865.0,43123.0,...,118952.0,698.0,0.0,1907.0,11.0,2777.0,347.0,335.0,4974.0,6314.0
ATLDMDBO,163308448.0,113468029.0,131798388.0,118641599.0,106301802.0,63769133.0,148292927.0,101468186.0,77986006.0,94320028.0,...,143545801.0,70368609.0,99495350.0,153804927.0,93762902.0,76888377.0,119375444.0,112349280.0,95457203.0,165713328.0


In [ ]:
hs4 = pd.read_csv('./hs4.csv')
hs4 = hs4.drop("2024수출액(달러)", axis=1)
hs4.columns = ["index", 'hs4', 'tag']

In [ ]:
result = []
for i in train["hs4"]:
    temp = hs4[i == hs4["hs4"]]
    if not temp.empty:
        result.append(temp["tag"].values[0])
    else:
        result.append("분류 불가")

In [ ]:
train["hs4_tag"] = result

In [ ]:
# min_ = 1
# max_ = 0
# for i in monthly["item_id"].unique():
#     temp = np.abs(monthly[monthly["item_id"] == i]["weight"].corr(monthly[monthly["item_id"] == i]["value"]))
#     print(temp)
#     if temp < min_:
#         min_ = temp
#     if temp > max_:

In [ ]:
monthly.pivot(index="item_id", columns="ym",values=["value","weight"]).fillna(0.0).T

item_id          AANGBULD  AHMDUILJ  ANWUJOKX  APQGTRMF     ATLDMDBO  \
       ym                                                              
value  2022-01    14276.0  242705.0       0.0  383999.0  143097177.0   
       2022-02    52347.0  120847.0       0.0  512813.0  103568323.0   
       2022-03    53549.0  197317.0       0.0  217064.0  118403737.0   
       2022-04        0.0  126142.0   63580.0  470398.0  121873741.0   
       2022-05    26997.0   71730.0   81670.0  539873.0  115024617.0   
...                   ...       ...       ...       ...          ...   
weight 2025-03        0.0   43332.0       0.0    2777.0   76888377.0   
       2025-04    49050.0   44913.0       0.0     347.0  119375444.0   
       2025-05        0.0   44035.0       0.0     335.0  112349280.0   
       2025-06   865246.0   25574.0       0.0    4974.0   95457203.0   
       2025-07  1046036.0   34463.0       0.0    6314.0  165713328.0   

item_id         AXULOHBQ   BEZYMBBT  BJALXPFS  BLANHGYY   BSRMSVTC  ...  \
       ym                                                           ...   
value  2022-01   11429.0  6034752.0  352532.0       0.0  1216269.0  ...   
       2022-02   16365.0  6221091.0  218947.0     891.0   156005.0  ...   
       2022-03   47307.0  4611537.0  331472.0       0.0   801085.0  ...   
       2022-04   59524.0  5202039.0   17480.0       0.0   365605.0  ...   
       2022-05   74828.0  6889516.0  234330.0   23141.0   168725.0  ...   
...                  ...        ...       ...       ...        ...  ...   
weight 2025-03    6678.0     5338.0   27832.0      15.0      901.0  ...   
       2025-04    9496.0     4594.0   66763.0      15.0     1356.0  ...   
       2025-05   13021.0     4008.0   62149.0      60.0      517.0  ...   
       2025-06    8195.0     3429.0  121446.0      12.0       30.0  ...   
       2025-07    2915.0     2540.0  158613.0     367.0     2714.0  ...   

item_id         XIFHSOWQ   XIIEJNEE  XIPPENFQ   XMKRPGLB    XUOIQPFL  \
       ym                                                              
value  2022-01    7718.0        0.0  108699.0   879207.0  10871069.0   
       2022-02   24829.0       81.0   78853.0  1650028.0   9573079.0   
       2022-03  102019.0  3198210.0  172237.0  1831614.0   8774783.0   
       2022-04   33215.0        0.0   89681.0   705567.0   4713153.0   
       2022-05   82348.0    81175.0   81387.0  1718634.0   4612292.0   
...                  ...        ...       ...        ...         ...   
weight 2025-03     544.0       25.0   55115.0  4947732.0   1035737.0   
       2025-04     730.0       50.0  323952.0  1666415.0   1380280.0   
       2025-05    2881.0    14200.0  127323.0  4474885.0   1134783.0   
       2025-06     748.0        0.0  190036.0  4223478.0    750994.0   
       2025-07    1187.0     1651.0  329843.0  1177973.0    675695.0   

item_id         YSYHGLQK  ZCELVYQU   ZGJXVMNI  ZKENOUDA  ZXERAXWP  
       ym                                                          
value  2022-01       0.0  373859.0  1154724.0  122627.0   10867.0  
       2022-02     543.0   59900.0  1337622.0   38162.0     675.0  
       2022-03     766.0   31158.0  1662893.0    3592.0   20548.0  
       2022-04    1108.0  594407.0  1561647.0    5613.0     662.0  
       2022-05     859.0  648232.0  1603223.0   33924.0   18949.0  
...                  ...       ...        ...       ...       ...  
weight 2025-03       7.0   30027.0     4098.0    1326.0    3507.0  
       2025-04       0.0   36070.0     4075.0    1562.0    4258.0  
       2025-05       0.0    9440.0     3906.0    1227.0    4598.0  
       2025-06       5.0   19100.0     4864.0   41324.0   36632.0  
       2025-07      24.0    5938.0     4764.0    1431.0    4777.0  

[86 rows x 100 columns]

In [ ]:
import pandas as pd

def aggregate_monthly(df: pd.DataFrame,
                      year_col: str = "year",
                      month_col: str = "month",
                      item_col: str = "item_id",
                      value_col: str = "value",
                      weight_col: str = "weight",
                      quantity_col: str = "quantity",
                      ensure_continuous_months: bool = False) -> pd.DataFrame:
    """
    동일 item_id를 년도별 동일 월로 합산하여
    <ITEM>_value, <ITEM>_weight, <ITEM>_quantity 컬럼으로 와이드 피벗한 결과를 반환합니다.

    Parameters
    ----------
    df : pd.DataFrame
        원본 데이터 (컬럼: item_id, year, month, value, weight, quantity 포함)
    ensure_continuous_months : bool
        True면 데이터의 최소~최대 ym 사이의 모든 달을 채워 결측을 0.0으로 채움

    Returns
    -------
    out : pd.DataFrame
        컬럼: ym, <ITEM>_value, <ITEM>_weight, <ITEM>_quantity ...
    """

    # 1) ym(YYYY-MM) 생성
    df = df.copy()
    df["ym"] = pd.to_datetime(dict(year=df[year_col], month=df[month_col], day=1)).dt.strftime("%Y-%m")

    # 2) ym × item_id별 합산
    agg = (
        df.groupby(["ym", item_col], as_index=False)
          .agg(
              weight_sum=(weight_col, "sum"),
              value_sum=(value_col, "sum"),
              quantity_sum=(quantity_col, "sum")
          )
    )

    # 3) 기준 ym 인덱스 정의
    if ensure_continuous_months:
        # 최소~최대 사이 모든 달 생성
        ym_dates = pd.to_datetime(agg["ym"] + "-01").sort_values().unique()
        start = ym_dates[0]
        end = ym_dates[-1]
        all_months_dt = pd.date_range(start=start, end=end, freq="MS")
        all_months = [d.strftime("%Y-%m") for d in all_months_dt]
    else:
        # 데이터에 존재하는 ym만
        all_months = sorted(agg["ym"].unique())

    # 4) 와이드 피벗 (value/weight/quantity → 각 item별 3컬럼)
    wide = (
        agg.pivot(index="ym", columns=item_col, values=["value_sum", "weight_sum", "quantity_sum"])
           .reindex(all_months)        # 지정한 ym 순서로 정렬
           .sort_index()
    )

    # 5) 결측치 0.0
    wide = wide.fillna(0.0)

    # 6) 컬럼 평탄화: "<ITEM>_value", "<ITEM>_weight", "<ITEM>_quantity"
    flat_cols = []
    for top, item in wide.columns.to_flat_index():
        if top == "value_sum":
            flat_cols.append(f"{item}_value")
        elif top == "weight_sum":
            flat_cols.append(f"{item}_weight")
        elif top == "quantity_sum":
            flat_cols.append(f"{item}_quantity")
        else:
            flat_cols.append(f"{item}_{top}")
    wide.columns = flat_cols

    # 7) 아이템별로 [value → weight → quantity] 순서 정렬
    def _tri_order(cols):
        from collections import defaultdict
        bucket = defaultdict(dict)
        for c in cols:
            if c.endswith("_value"):
                bucket[c[:-6]]["v"] = c
            elif c.endswith("_weight"):
                bucket[c[:-7]]["w"] = c
            elif c.endswith("_quantity"):
                bucket[c[:-9]]["q"] = c
        ordered = []
        for k in sorted(bucket.keys()):  # item_id 알파벳순 정렬 (원하시면 커스텀 가능)
            if "v" in bucket[k]: ordered.append(bucket[k]["v"])
            if "w" in bucket[k]: ordered.append(bucket[k]["w"])
            if "q" in bucket[k]: ordered.append(bucket[k]["q"])
        return ordered

    wide = wide[_tri_order(wide.columns)]

    # 8) ym을 컬럼으로 복원
    out = wide.reset_index()

    return out

In [ ]:
out_df = aggregate_monthly(train, ensure_continuous_months=False)

In [ ]:
import plotly.express as px

# train의 앞 5개 열만 사용하여 이름 지정
# df = pivot.T.reset_index().iloc[:, 0:5].copy()
df = pivot.T.reset_index()
# df.columns = ["ym", "AANGBULD", "AHMDUILJ", "ANWUJOKX", "APQGTRMF"]  # R의 structure(names=...)와 동일

# pivot_longer() → melt()
lme_long = df.melt(id_vars="ym", var_name="Metal", value_name="value")

# ggplot(line) → plotly line
fig = px.line(
    lme_long,
    x="ym",
    y="value",
    color="Metal",
    # title="LME : Closing Price of Industrial Metals",
    labels={"date_id": "Date (date_id)", "value": ""}
)

# R의 figsize(16, 6) 비슷하게 크기 조정 (픽셀 단위)
fig.update_layout(width=1100, height=400, legend_title_text="Metal", margin=dict(l=40, r=20, t=60, b=40))

fig.show()

In [4]:
df = pivot.T.reset_index()
scaler = MinMaxScaler()
# scaler = StandardScaler()
df_minmax = pd.DataFrame(scaler.fit_transform(df.drop("ym",axis=1)),columns=df.drop("ym",axis=1).columns).set_index(df["ym"]).reset_index()

In [5]:
df_vw = pivot_w.T.reset_index()
scaler_vw = MinMaxScaler()
# scaler = StandardScaler()
df_minmax_vw = pd.DataFrame(scaler_vw.fit_transform(df_vw.drop("ym",axis=1)),columns=df_vw.drop("ym",axis=1).columns).set_index(df_vw["ym"]).reset_index()

In [7]:
df_minmax_vw

item_id,ym,AANGBULD,AHMDUILJ,ANWUJOKX,APQGTRMF,ATLDMDBO,AXULOHBQ,BEZYMBBT,BJALXPFS,BLANHGYY,...,XIFHSOWQ,XIIEJNEE,XIPPENFQ,XMKRPGLB,XUOIQPFL,YSYHGLQK,ZCELVYQU,ZGJXVMNI,ZKENOUDA,ZXERAXWP
0,2022-01,0.016849,1.000000,0.000000,0.421960,0.976410,0.003063,0.798254,0.336436,0.0000,...,0.000000,0.000000,0.091180,0.338734,1.000000,0.000000,0.380084,0.000000,0.003920,0.058549
1,2022-02,0.064991,0.418210,0.000000,0.684553,0.487511,0.000000,0.687247,0.635654,0.0288,...,0.018240,0.000000,0.198996,0.803000,0.823894,0.119403,0.053349,0.090703,0.002396,0.001038
2,2022-03,0.066483,0.627174,0.000000,0.364096,0.667319,0.019964,0.427814,1.000000,0.0000,...,0.013380,1.000000,0.497292,0.871618,0.695160,0.283582,0.317165,0.133906,0.000073,0.119883
3,2022-04,0.000000,0.410051,0.750100,0.525464,0.538260,0.066421,0.569068,0.003147,0.0000,...,0.000980,0.000000,0.032500,0.236510,0.197199,0.343284,1.000000,0.143859,0.000266,0.000055
4,2022-05,0.032669,0.196033,0.992096,0.711884,0.417215,0.056587,0.883536,0.529436,0.0496,...,0.012910,0.001809,0.046846,0.775913,0.144324,0.253731,0.849306,0.079620,0.001960,0.076108
5,2022-06,0.099104,0.579768,0.347249,0.314623,0.000000,0.022382,0.666823,0.028241,0.0032,...,0.001940,0.000020,0.049066,0.877083,0.041384,0.388060,0.268768,0.158109,0.004743,0.001748
6,2022-07,0.000000,0.601636,0.115791,0.963414,0.829118,0.050004,0.951824,0.116221,0.0416,...,0.019917,0.000000,0.110996,0.132987,0.420674,0.149254,0.233553,0.204931,0.485239,0.100959
7,2022-08,0.000000,0.622240,0.000000,0.330427,0.369801,0.083403,0.545058,0.075742,0.0080,...,0.004895,0.000000,0.196440,0.270060,0.310957,0.164179,0.000000,0.116716,0.486860,0.002376
8,2022-09,0.000000,0.598178,0.000000,0.881574,0.139457,0.461456,0.863580,0.272956,0.0080,...,0.001049,0.000000,0.399991,0.287922,0.265441,0.000000,0.322727,0.129156,0.024320,0.010131
9,2022-10,0.000000,0.185053,1.000000,0.362524,0.299683,1.000000,0.948550,0.767562,0.0752,...,0.028885,0.000000,0.150555,0.192424,0.165787,0.447761,0.064964,0.109251,0.002347,0.051312


In [8]:
import plotly.express as px

uss_df = df_minmax_vw.copy()

# pivot_longer → melt
uss_long = uss_df.melt(id_vars="ym", var_name="item", value_name="value")

# ggplot(line) + facet_wrap(~ Stock, ncol=6) → plotly line + facet_col_wrap
fig = px.line(
    uss_long,
    x="ym",
    y="value",
    color="item",
    facet_col="item",
    facet_col_wrap=6,
    facet_row_spacing=0.02,   # ★ 세로 간격 축소
    color_discrete_sequence=["#2962FF"],
)

# 패싯 라벨 정리 & 범례 제거 & 레이아웃 크기 조정
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width=1800, height=2100, showlegend=False)

# 주의: y축을 0~150으로 제한해 대부분의 티커 가독성을 높였습니다. 고가 티커는 전체 범위가 보이지 않을 수 있습니다
fig.show()

In [9]:
import plotly.express as px

uss_df = df_minmax.copy()

# # R의 structure(names = c("date_id", uss_tickers))와 동일
# uss_df.columns = ["date_id"] + list(uss_tickers)

# pivot_longer → melt
uss_long = uss_df.melt(id_vars="ym", var_name="item", value_name="value")

# ggplot(line) + facet_wrap(~ Stock, ncol=6) → plotly line + facet_col_wrap
fig = px.line(
    uss_long,
    x="ym",
    y="value",
    color="item",
    facet_col="item",
    facet_col_wrap=6,
    facet_row_spacing=0.02,   # ★ 세로 간격 축소
    color_discrete_sequence=["#2962FF"],
)

# 패싯 라벨 정리 & 범례 제거 & 레이아웃 크기 조정
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width=1800, height=2100, showlegend=False)

# 주의: y축을 0~150으로 제한해 대부분의 티커 가독성을 높였습니다. 고가 티커는 전체 범위가 보이지 않을 수 있습니다
fig.show()

In [10]:
import pandas as pd
import plotly.express as px

# 0) 공통 전제: 두 DF 모두 첫 컬럼이 'ym', 나머지가 item 컬럼들
#    (예: df_minmax.columns = ['ym', 'A', 'B', ...])
#    x축(ym) 월 구성이 같다고 가정

# 1) 길게 만들기 + 출처(series) 라벨 부여
long_a = (
    df_minmax
      .melt(id_vars="ym", var_name="item", value_name="value")
      .assign(series="minmax")
)
long_b = (
    df_minmax_vw
      .melt(id_vars="ym", var_name="item", value_name="value")
      .assign(series="minmax_vw")
)

# 2) (선택) 공통 item만 사용하고 싶다면 inner merge 대신 교집합 필터
common_items = sorted(set(long_a["item"]).intersection(set(long_b["item"])))
long_a = long_a[long_a["item"].isin(common_items)]
long_b = long_b[long_b["item"].isin(common_items)]

# 3) 합치기
plot_df = pd.concat([long_a, long_b], ignore_index=True)

# 4) 라인 플롯: 패싯은 item, 색상/스타일은 series(두 개 라인)
fig = px.line(
    plot_df,
    x="ym",
    y="value",
    color="series",           # 각 패싯에 'minmax' vs 'minmax_vw'
    line_dash="series",       # 두 라인의 dash를 달리해서 더 잘 구분
    facet_col="item",
    facet_col_wrap=6,
    facet_row_spacing=0.02,
    # 원하시는 색이 있다면 아래 맵을 쓰세요 (없으면 Plotly 기본 팔레트 사용)
    color_discrete_map={
        "minmax": "#2962FF",
        "minmax_vw": "#3B3A3A",
    },
    category_orders={"series": ["minmax", "minmax_vw"]},
)

# 5) 패싯 제목 간소화(=뒤 텍스트만 남기기) + 레이아웃
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width=1800, height=2100, legend_title_text="Series", showlegend=True)

# 6) (선택) y축 범위 고정
#    두 시계열이 동일 스케일이라면 아래처럼 고정 가능
# for ax in fig.layout:
#     if ax.startswith("yaxis"):
#         fig.layout[ax].update(range=[0, 150])

# 7) (선택) hover 깔끔하게
fig.update_traces(hovertemplate="ym=%{x}<br>%{legendgroup}: %{y:.2f}")

fig.show()

## 3. 공행성쌍 탐색
- 각 (A, B) 쌍에 대해 lag = 1 ~ max_lag까지 Pearson 상관계수 계산
- 절댓값이 가장 큰 상관계수와 lag를 선택
- |corr| >= corr_threshold이면 A→B 공행성 있다고 판단

In [6]:
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(pivot, max_lag=6, min_nonzero=12, corr_threshold=0.4):
    '''
    pivot = 상관관계를 비교할 column 리스트
    max_lag = 지연일
    min_nonzero = 무역량이 존재하는 달의 최소 수치
    corr_threshold = 상관계수 임계값
    '''
    items = pivot.index.to_list()                           # item_id 값 추출
    months = pivot.columns.to_list()                        # 시간 정보 추출
    n_months = len(months)                                  # 시간 정보 최대 길이
    results = []                                            # 결과 저장 용 빈 리스트
    
    for i, leader in tqdm(enumerate(items)):                # 단일 item_id 별 반복
        x = pivot.loc[leader].values.astype(float)          # 단일 item 시간 별 무역량
        if np.count_nonzero(x) < min_nonzero:               # 무역량이 존재하는 달의 총합 수가 지정한 최소 수치(min_nonzero)를 넘는 지 판별
            continue

        for follower in items:                              # target_item을 제외한 다른 item과 corr 비교
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)    # 비교할 item의 무역량 최소 수치 판별
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None                                 # corr이 제일 높게 나오는 지연일
            best_corr = 0.0                                 # 제일 높은 corr

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):               # 1일 차이부터 max_lag까지 corr 비교
                if n_months <= lag:                         # 아마도 max_lag 값 오입력 할 경우 예외 처리
                    print("이게 통과하는 경우가 있어?")
                    continue
                corr = safe_corr(x[:-lag], y[lag:]) 
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)                           # 반환을 위한 데이터프레임화
    return pairs

def find_comovement_multi_pairs(pivot_v, pivot_w, max_lag=6, min_nonzero=12, corr_threshold=0.4):
    '''
    pivot = 상관관계를 비교할 column 리스트
    max_lag = 지연일
    min_nonzero = 무역량이 존재하는 달의 최소 수치
    corr_threshold = 상관계수 임계값
    '''
    items = pivot_v.index.to_list()                           # item_id 값 추출
    months = pivot_v.columns.to_list()                        # 시간 정보 추출
    n_months = len(months)                                  # 시간 정보 최대 길이
    results = []                                            # 결과 저장 용 빈 리스트
    
    for i, leader in tqdm(enumerate(items)):                # 단일 item_id 별 반복
        x = pivot_v.loc[leader].values.astype(float)          # 단일 item 시간 별 무역량
        x_w = pivot_w.loc[leader].values.astype(float) 
        if np.count_nonzero(x) < min_nonzero:               # 무역량이 존재하는 달의 총합 수가 지정한 최소 수치(min_nonzero)를 넘는 지 판별
            continue

        for follower in items:                              # target_item을 제외한 다른 item과 corr 비교
            if follower == leader:
                continue

            y = pivot_v.loc[follower].values.astype(float)    # 비교할 item의 무역량 최소 수치 판별
            y_w = pivot_w.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None                                 # corr이 제일 높게 나오는 지연일
            best_corr = 0.0                                 # 제일 높은 corr

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):               # 1일 차이부터 max_lag까지 corr 비교
                if n_months <= lag:                         # 아마도 max_lag 값 오입력 할 경우 예외 처리
                    print("이게 통과하는 경우가 있어?")
                    continue
                # corr = safe_corr(x[:-lag], y[lag:]) 
                corr_v = safe_corr(x[:-lag], y[lag:])
                corr_w = safe_corr(x_w[:-lag], y_w[lag:])
                corr = (corr_v + corr_w)/2
                # corr = np.array([corr_v, corr_w, (corr_v + corr_w)/2])
                # print(corr)
                # if np.argmax(np.abs(corr)) == 1:
                #     print(corr)
                # corr = corr[np.argmax(np.abs(corr))]
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)                           # 반환을 위한 데이터프레임화
    return pairs

pairs2 = find_comovement_pairs(pivot, max_lag=6, min_nonzero=29, corr_threshold=0.3)

print("탐색된 공행성쌍 수:", len(pairs2))
pairs2.head()

0it [00:00, ?it/s]

100it [00:06, 16.49it/s]

탐색된 공행성쌍 수: 2951


,leading_item_id,following_item_id,best_lag,max_corr
0,AANGBULD,APQGTRMF,5,-0.443984
1,AANGBULD,BEZYMBBT,1,-0.333863
2,AANGBULD,DDEXPPXU,2,0.383169
3,AANGBULD,DEWLVASR,6,0.640221
4,AANGBULD,DNMPSKTB,4,-0.410635


In [9]:
import numpy as np
import pandas as pd
from scipy import stats

# -----------------------------
# 0) 하이퍼파라미터
# -----------------------------
MAX_LAG = 6                 # 탐색할 최대 지연월
MIN_NONZERO = 12            # 각 시계열에서 0이 아닌 달 최소 개수
MIN_OVERLAP = 24            # 랙 적용 후 실제 겹치는 구간 최소 길이
CORR_THRESH = 0.40          # 상관계수 절대값 임계치
ALPHA = 0.05                # 유의수준(신뢰구간/검정용)
USE_SPEARMAN = False        # True면 Spearman, False면 Pearson

# -----------------------------
# 1) 전처리 유틸
# -----------------------------
def winsorize(a, q=0.01):
    lo, hi = np.quantile(a[np.isfinite(a)], [q, 1-q])
    return np.clip(a, lo, hi)

def transform_series(v, mode="log1p-pct", winsor=0.01, zscore=True):
    """
    mode:
      - "level"        : 원자료
      - "log1p"        : log1p 변환
      - "log1p-pct"    : log1p 후 pct_change
      - "diff"         : 1차 차분
      - "diff12"       : 12차 계절차분
    """
    x = v.astype(float).copy()
    if mode == "log1p":
        x = np.log1p(x)
    elif mode == "log1p-pct":
        x = np.log1p(x)
        x = pd.Series(x).pct_change().to_numpy()
    elif mode == "diff":
        x = pd.Series(x).diff().to_numpy()
    elif mode == "diff12":
        x = pd.Series(x).diff(12).to_numpy()
    # 윈저라이즈
    # if winsor:
    #     x = winsorize(x, q=winsor)
    # 표준화
    if zscore:
        m, s = np.nanmean(x), np.nanstd(x)
        if s > 0:
            x = (x - m) / s
    # 초기 변환으로 생긴 NaN은 제거 대신 0으로 채우되, 겹침 판단은 별도로 함
    x = np.where(np.isfinite(x), x, 0.0)
    return x

def lag_overlap_corr(x, y, lag, spearman=False):
    """
    lag>0: x가 선행(leader), y가 lag만큼 후행(follower)
    """
    if lag <= 0 or lag >= len(x):
        return np.nan, 0
    xw = x[:-lag]
    yw = y[lag:]
    # 실제 겹치는 유효 길이(둘 다 finite)
    # mask = np.isfinite(xw) & np.isfinite(yw)
    # xw = xw[mask]; yw = yw[mask]
    n = len(xw)
    if n < 2:
        return np.nan, 0
    if spearman:
        r, _ = stats.spearmanr(xw, yw)
    else:
        # 표준편차 0 방지
        if np.std(xw) == 0 or np.std(yw) == 0:
            return 0.0, n
        r = float(np.corrcoef(xw, yw)[0, 1])
    return r, n

def ar1_lag1_autocorr(x):
    """간단한 AR(1) 계수 추정(자유도 최소화)."""
    x = pd.Series(x).dropna().to_numpy()
    if len(x) < 3:
        return 0.0
    x1, x2 = x[:-1], x[1:]
    if np.std(x1)==0 or np.std(x2)==0:
        return 0.0
    return float(np.corrcoef(x1, x2)[0, 1])

def fisher_ci(r, n_eff, alpha=0.05):
    """Fisher z 변환 기반 신뢰구간."""
    r = np.clip(r, -0.999999, 0.999999)
    if n_eff <= 3:
        return -1.0, 1.0
    z = np.arctanh(r)
    se = 1/np.sqrt(n_eff-3)
    zc = stats.norm.ppf(1-alpha/2)
    lo, hi = np.tanh(z - zc*se), np.tanh(z + zc*se)
    return lo, hi

def effective_sample_size(n, rho_x1, rho_y1):
    """자기상관 보정된 유효 표본 크기(N_eff)의 근사."""
    num = n*(1 - rho_x1)*(1 - rho_y1)
    den = (1 + rho_x1)*(1 + rho_y1)
    if den <= 0:  # 수치 안정화
        den = 1e-6
    return max(num/den, 2)

# -----------------------------
# 2) 메인: 공행성(lead–lag) 페어 탐색
# -----------------------------
def find_leadlag_pairs(
    pivot, max_lag=MAX_LAG, min_nonzero=MIN_NONZERO, min_overlap=MIN_OVERLAP,
    corr_threshold=CORR_THRESH, alpha=ALPHA, transform="log1p-pct",
    use_spearman=USE_SPEARMAN
):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    nT = len(months)

    # 사전 전처리 및 통계 준비
    series = {}
    ar1 = {}
    nonzero_cnt = {}
    raw = pivot.values  # (n_item, nT)

    for i, item in enumerate(items):
        v_raw = raw[i, :].astype(float)
        nonzero_cnt[item] = int(np.count_nonzero(v_raw))
        x = transform_series(v_raw, mode=transform,zscore=False)
        series[item] = x
        ar1[item] = ar1_lag1_autocorr(x)
    # print(ar1)

    results = []
    for i, leader in enumerate(items):
        if nonzero_cnt[leader] < min_nonzero:
            continue
        x = series[leader]
        for j, follower in enumerate(items):
            if i == j:
                continue
            if nonzero_cnt[follower] < min_nonzero:
                continue
            y = series[follower]

            best = {"lag": None, "r": 0.0, "n": 0}
            # lag=1..max_lag 탐색 (leader 선행)
            for lag in range(1, max_lag+1):
                r, n = lag_overlap_corr(x, y, lag, spearman=use_spearman)
                if n < min_overlap:
                    continue
                if np.isnan(r):
                    continue
                if abs(r) > abs(best["r"]):
                    best = {"lag": lag, "r": r, "n": n}

            if best["lag"] is None:
                continue

            # 유효 표본 보정 후 신뢰구간
            n_eff = effective_sample_size(best["n"], ar1[leader], ar1[follower])
            lo, hi = fisher_ci(best["r"], n_eff, alpha=alpha)
            # 임계값/유의성 필터
            # if abs(lo-hi) > 0.5:
            if abs(best["r"]) >= corr_threshold:
            # if abs(best["r"]) >= corr_threshold and not (lo <= 0 <= hi):
            # if abs(best["r"]) >= corr_threshold and (lo <= 0 <= hi):
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best["lag"],
                    "max_corr": best["r"],
                    "overlap_n": int(best["n"]),
                    "n_eff": float(n_eff),
                    "ci_low": float(lo),
                    "ci_high": float(hi),
                    "ci_diff": abs(float(lo) - float(hi)),
                })
    pairs = pd.DataFrame(results)
    # pairs = pd.DataFrame(results).sort_values(["max_corr"], ascending=False, ignore_index=True)
    return pairs

# -----------------------------
# 3) 실행 예시
# -----------------------------
# 위에서 이미 pivot을 생성하셨으므로 바로 호출:
pairs = find_leadlag_pairs(
    pivot,
    max_lag=6,
    min_nonzero=29,
    min_overlap=24,
    corr_threshold=0.3,
    alpha=0.05,
    # transform="log1p",   # 추천: 규모/추세 영향 완화
    transform="level",
    use_spearman=False       # 필요시 True로 바꿔 비선형 순위관계 테스트
)

print("탐색된 공행성쌍 수:", len(pairs))
pairs.head()

탐색된 공행성쌍 수: 2951


,leading_item_id,following_item_id,best_lag,max_corr,overlap_n,n_eff,ci_low,ci_high,ci_diff
0,AANGBULD,APQGTRMF,5,-0.443984,38,5.477658,-0.938145,0.645756,1.583901
1,AANGBULD,BEZYMBBT,1,-0.333863,42,9.900307,-0.798079,0.379059,1.177137
2,AANGBULD,DDEXPPXU,2,0.383169,41,12.757334,-0.220029,0.774399,0.994427
3,AANGBULD,DEWLVASR,6,0.640221,37,4.512650,-0.683178,0.982050,1.665228
4,AANGBULD,DNMPSKTB,4,-0.410635,39,2.000000,-1.000000,1.000000,2.000000


In [10]:
# 아이템 별 pair 갯수 -> 페어 item id를 워한다면 len() 제거
pairs_count = {}
for i in pairs.leading_item_id.unique():
    pairs_count[i] = len(pairs[pairs["leading_item_id"] == i]["following_item_id"].values)
pairs_count


{'AANGBULD': 35,
 'AHMDUILJ': 33,
 'APQGTRMF': 46,
 'ATLDMDBO': 51,
 'AXULOHBQ': 40,
 'BEZYMBBT': 43,
 'BJALXPFS': 40,
 'BLANHGYY': 38,
 'BSRMSVTC': 35,
 'BTMOEMEP': 44,
 'BUZIIBYG': 23,
 'CCLHWFWF': 33,
 'DBWLZWNK': 47,
 'DDEXPPXU': 18,
 'DEWLVASR': 46,
 'DNMPSKTB': 49,
 'DUCMGGNW': 23,
 'ELQGMQWE': 45,
 'EVBVXETX': 36,
 'FCYBOAXC': 25,
 'FDXPMYGF': 27,
 'FITUEHWN': 21,
 'FQCLOEXA': 38,
 'FRHNWLNI': 35,
 'FTSVTTSR': 20,
 'FWUCPMMW': 23,
 'GKQIJYDH': 19,
 'GYHKIVQT': 53,
 'HCDTGMST': 30,
 'HXYSSRXE': 48,
 'IGDVVKUD': 42,
 'JBVHSUWY': 33,
 'JERHKLYW': 33,
 'JPBRUTWP': 51,
 'JSLXRQOK': 19,
 'KAGJCHMR': 30,
 'KEUWZRKO': 27,
 'KJNSOAHR': 30,
 'LLHREMKS': 25,
 'LPHPPJUG': 37,
 'LRVGFDFM': 43,
 'LSOIUSXD': 45,
 'LTOYKIML': 25,
 'LUENUFGA': 30,
 'MBSBZBXA': 24,
 'MIRCVAMV': 22,
 'NZKBIBNU': 34,
 'OGAFEHLU': 46,
 'OJIFIHMZ': 21,
 'OKMBFVKS': 47,
 'OXKURKXR': 41,
 'QJQJSWFU': 35,
 'QKXNTIIB': 18,
 'QRKRBYJL': 50,
 'QVLMOEYE': 41,
 'RAWUKQMJ': 36,
 'RCBZUSIM': 46,
 'RJGPVEXX': 42,
 'ROACSLMG': 3

In [13]:
# 아이템 별 pair 갯수 -> 페어 item id를 워한다면 len() 제거
pairs2_count = {}
for i in pairs2.leading_item_id.unique():
    pairs2_count[i] = len(pairs2[pairs2["leading_item_id"] == i]["following_item_id"].values)
pairs2_count

{'AANGBULD': 35,
 'AHMDUILJ': 33,
 'APQGTRMF': 46,
 'ATLDMDBO': 51,
 'AXULOHBQ': 40,
 'BEZYMBBT': 43,
 'BJALXPFS': 40,
 'BLANHGYY': 38,
 'BSRMSVTC': 35,
 'BTMOEMEP': 44,
 'BUZIIBYG': 23,
 'CCLHWFWF': 33,
 'DBWLZWNK': 47,
 'DDEXPPXU': 18,
 'DEWLVASR': 46,
 'DNMPSKTB': 49,
 'DUCMGGNW': 23,
 'ELQGMQWE': 45,
 'EVBVXETX': 36,
 'FCYBOAXC': 25,
 'FDXPMYGF': 27,
 'FITUEHWN': 21,
 'FQCLOEXA': 38,
 'FRHNWLNI': 35,
 'FTSVTTSR': 20,
 'FWUCPMMW': 23,
 'GKQIJYDH': 19,
 'GYHKIVQT': 53,
 'HCDTGMST': 30,
 'HXYSSRXE': 48,
 'IGDVVKUD': 42,
 'JBVHSUWY': 33,
 'JERHKLYW': 33,
 'JPBRUTWP': 51,
 'JSLXRQOK': 19,
 'KAGJCHMR': 30,
 'KEUWZRKO': 27,
 'KJNSOAHR': 30,
 'LLHREMKS': 25,
 'LPHPPJUG': 37,
 'LRVGFDFM': 43,
 'LSOIUSXD': 45,
 'LTOYKIML': 25,
 'LUENUFGA': 30,
 'MBSBZBXA': 24,
 'MIRCVAMV': 22,
 'NZKBIBNU': 34,
 'OGAFEHLU': 46,
 'OJIFIHMZ': 21,
 'OKMBFVKS': 47,
 'OXKURKXR': 41,
 'QJQJSWFU': 35,
 'QKXNTIIB': 18,
 'QRKRBYJL': 50,
 'QVLMOEYE': 41,
 'RAWUKQMJ': 36,
 'RCBZUSIM': 46,
 'RJGPVEXX': 42,
 'ROACSLMG': 3

In [14]:
# 아이템 별 pair 갯수 -> 페어 item id를 워한다면 len() 제거
sub2_count = {}
for i in sub2.leading_item_id.unique():
    sub2_count[i] = len(sub2[sub2["leading_item_id"] == i]["following_item_id"].values)
sub2_count

{'AANGBULD': 35,
 'AHMDUILJ': 33,
 'APQGTRMF': 46,
 'ATLDMDBO': 51,
 'AXULOHBQ': 40,
 'BEZYMBBT': 43,
 'BJALXPFS': 40,
 'BLANHGYY': 38,
 'BSRMSVTC': 35,
 'BTMOEMEP': 44,
 'BUZIIBYG': 23,
 'CCLHWFWF': 33,
 'DBWLZWNK': 47,
 'DDEXPPXU': 18,
 'DEWLVASR': 46,
 'DNMPSKTB': 49,
 'DUCMGGNW': 23,
 'ELQGMQWE': 45,
 'EVBVXETX': 36,
 'FCYBOAXC': 25,
 'FDXPMYGF': 27,
 'FITUEHWN': 21,
 'FQCLOEXA': 38,
 'FRHNWLNI': 35,
 'FTSVTTSR': 20,
 'FWUCPMMW': 23,
 'GKQIJYDH': 19,
 'GYHKIVQT': 53,
 'HCDTGMST': 30,
 'HXYSSRXE': 48,
 'IGDVVKUD': 42,
 'JBVHSUWY': 33,
 'JERHKLYW': 33,
 'JPBRUTWP': 51,
 'JSLXRQOK': 19,
 'KAGJCHMR': 30,
 'KEUWZRKO': 27,
 'KJNSOAHR': 30,
 'LLHREMKS': 25,
 'LPHPPJUG': 37,
 'LRVGFDFM': 43,
 'LSOIUSXD': 45,
 'LTOYKIML': 25,
 'LUENUFGA': 30,
 'MBSBZBXA': 24,
 'MIRCVAMV': 22,
 'NZKBIBNU': 34,
 'OGAFEHLU': 46,
 'OJIFIHMZ': 21,
 'OKMBFVKS': 47,
 'OXKURKXR': 41,
 'QJQJSWFU': 35,
 'QKXNTIIB': 18,
 'QRKRBYJL': 50,
 'QVLMOEYE': 41,
 'RAWUKQMJ': 36,
 'RCBZUSIM': 46,
 'RJGPVEXX': 42,
 'ROACSLMG': 3

In [134]:
# pairs = find_comovement_multi_pairs(pivot, pivot_w, max_lag=1, min_nonzero=0, corr_threshold=0.3)
# print("탐색된 공행성쌍 수:", len(pairs))
# pairs.head()

In [135]:
pairs["leading_item_id"].unique() # 무역량이 최소 6개 초과 존재하는 item 리스트

array(['AANGBULD', 'AHMDUILJ', 'APQGTRMF', 'ATLDMDBO', 'AXULOHBQ',
       'BEZYMBBT', 'BJALXPFS', 'BLANHGYY', 'BSRMSVTC', 'BTMOEMEP',
       'BUZIIBYG', 'CCLHWFWF', 'DBWLZWNK', 'DDEXPPXU', 'DEWLVASR',
       'DNMPSKTB', 'DUCMGGNW', 'ELQGMQWE', 'EVBVXETX', 'FCYBOAXC',
       'FDXPMYGF', 'FITUEHWN', 'FQCLOEXA', 'FRHNWLNI', 'FTSVTTSR',
       'FWUCPMMW', 'GKQIJYDH', 'GYHKIVQT', 'HCDTGMST', 'HXYSSRXE',
       'IGDVVKUD', 'JBVHSUWY', 'JERHKLYW', 'JPBRUTWP', 'JSLXRQOK',
       'KAGJCHMR', 'KEUWZRKO', 'KJNSOAHR', 'LLHREMKS', 'LPHPPJUG',
       'LRVGFDFM', 'LSOIUSXD', 'LTOYKIML', 'LUENUFGA', 'MBSBZBXA',
       'MIRCVAMV', 'NZKBIBNU', 'OGAFEHLU', 'OJIFIHMZ', 'OKMBFVKS',
       'OXKURKXR', 'QJQJSWFU', 'QKXNTIIB', 'QRKRBYJL', 'QVLMOEYE',
       'RAWUKQMJ', 'RCBZUSIM', 'RJGPVEXX', 'ROACSLMG', 'SAAYMURU',
       'SAHWCZNH', 'SDWAYPIK', 'SNHYOVBM', 'STZDBITS', 'SUOYXCHP',
       'UGEQLMXM', 'UIFPPCLR', 'UQYUIVVR', 'UXSPKBJR', 'VBYCLTYZ',
       'VMAQSTJE', 'VUAFAIYJ', 'VWMBASNE', 'WBLJNPZQ', 'WHPUAO

In [16]:
c, p, n, d = count_p(pairs, sub2)
p, n, d

(0, 0, 0)

In [138]:
c

{'AANGBULD': [37, 35, 34, 3, 1],
 'AHMDUILJ': [32, 33, 32, 0, 1],
 'APQGTRMF': [46, 46, 46, 0, 0],
 'ATLDMDBO': [50, 51, 50, 0, 1],
 'AXULOHBQ': [39, 40, 37, 2, 3],
 'BEZYMBBT': [43, 43, 42, 1, 1],
 'BJALXPFS': [40, 40, 40, 0, 0],
 'BLANHGYY': [38, 38, 35, 3, 3],
 'BSRMSVTC': [36, 35, 32, 4, 3],
 'BTMOEMEP': [45, 44, 44, 1, 0],
 'BUZIIBYG': [22, 23, 19, 3, 4],
 'CCLHWFWF': [33, 33, 32, 1, 1],
 'DBWLZWNK': [47, 47, 47, 0, 0],
 'DDEXPPXU': [19, 18, 18, 1, 0],
 'DEWLVASR': [44, 46, 44, 0, 2],
 'DNMPSKTB': [49, 49, 49, 0, 0],
 'DUCMGGNW': [20, 23, 19, 1, 4],
 'ELQGMQWE': [46, 45, 44, 2, 1],
 'EVBVXETX': [35, 36, 35, 0, 1],
 'FCYBOAXC': [25, 25, 25, 0, 0],
 'FDXPMYGF': [26, 27, 25, 1, 2],
 'FITUEHWN': [23, 21, 21, 2, 0],
 'FQCLOEXA': [39, 38, 37, 2, 1],
 'FRHNWLNI': [33, 35, 33, 0, 2],
 'FTSVTTSR': [23, 20, 17, 6, 3],
 'FWUCPMMW': [24, 23, 23, 1, 0],
 'GKQIJYDH': [19, 19, 18, 1, 1],
 'GYHKIVQT': [52, 53, 51, 1, 2],
 'HCDTGMST': [38, 30, 29, 9, 1],
 'HXYSSRXE': [49, 48, 48, 1, 0],
 'IGDVVKUD

In [139]:
REF_ITEM = "AANGBULD" 
target_col = np.append(["ym",REF_ITEM],pairs[pairs["leading_item_id"] == REF_ITEM]["following_item_id"].values)
target_col_sub = np.append(["ym",REF_ITEM],sub2[sub2["leading_item_id"] == REF_ITEM]["following_item_id"].values)
len(target_col)-2

37

In [140]:
temp_inter = np.intersect1d(target_col,target_col_sub,)
# temp_inter = np.append(temp_inter)
len(temp_inter)-2

34

In [141]:
temp_tar = np.setdiff1d(target_col,target_col_sub,)
temp_tar = np.append(["ym",REF_ITEM],temp_tar)
len(temp_tar)-2

3

In [142]:
temp_tar_sub = np.setdiff1d(target_col_sub,target_col,)
temp_tar_sub = np.append(["ym",REF_ITEM],temp_tar_sub)
len(temp_tar_sub)-2

1

In [143]:
visualize_(df_minmax, pairs, REF_ITEM, temp_inter)

In [144]:
visualize_(df_minmax, pairs, REF_ITEM, temp_tar)

In [145]:
visualize_(df_minmax, pairs, REF_ITEM, temp_tar_sub)

In [ ]:
# 1) 기준 아이템 설정
# REF_ITEM = "AHMDUILJ"   # 예: "DXY" 또는 uss_tickers[0]
target_col = np.append(["ym",REF_ITEM],pairs[pairs["leading_item_id"] == REF_ITEM]["following_item_id"].values)
# target_col = temp_tar
# target_col = df_minmax.columns
uss_df = df_minmax[target_col].copy()

# # R의 structure(names = c("date_id", uss_tickers))와 동일
# uss_df.columns = ["date_id"] + list(uss_tickers)

# pivot_longer → melt
uss_long = uss_df.melt(id_vars="ym", var_name="item", value_name="value")

# 2) 기준 시계열 추출
bench = (
    uss_long.loc[uss_long["item"] == REF_ITEM, ["ym", "value"]]
            .rename(columns={"value": "bench_value"})
)

# 3) 모든 아이템에 기준값 머지
dfc = uss_long.merge(bench, on="ym", how="left")

# 4) '자기 자신' 라인과 '기준' 라인 두 벌로 쌓기
plot_df = pd.concat(
    [
        dfc.assign(series="self",  val=dfc["value"]),
        dfc.assign(series=f"benchmark: {REF_ITEM}", val=dfc["bench_value"])
    ],
    ignore_index=True
)

# 5) 그리기: 패싯은 item으로, 색은 series로
fig = px.line(
    plot_df,
    x="ym",
    y="val",
    color="series",
    facet_col="item",
    facet_col_wrap=6,
    facet_row_spacing=0.02,
    # 원하시면 색을 고정해 가독성을 높일 수 있습니다.
    color_discrete_map={
        "self": "#2962FF",                 # 각 아이템(자기 자신)
        f"benchmark: {REF_ITEM}": "#9E9E9E"  # 기준 라인(회색)
    }
)

# 패싯 라벨 정리 & 범례/레이아웃
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]),)
fig.update_traces(opacity=0.95)
# fig.update_layout(width=1800, height=1400, showlegend=True)
fig.update_layout(showlegend=True,)

# 필요하면 Y축 범위 고정(주의: 기준과 스케일이 다르면 왜곡될 수 있음)
# fig.update_yaxes(range=[0, 150])

fig.show()

In [ ]:
# 1) 기준 아이템 설정
# REF_ITEM = "AHMDUILJ"   # 예: "DXY" 또는 uss_tickers[0]
target_col_sub = np.append(["ym",REF_ITEM],sub2[sub2["leading_item_id"] == REF_ITEM]["following_item_id"].values)
# target_col = df_minmax.columns
uss_df = df_minmax[target_col_sub].copy()

# # R의 structure(names = c("date_id", uss_tickers))와 동일
# uss_df.columns = ["date_id"] + list(uss_tickers)

# pivot_longer → melt
uss_long = uss_df.melt(id_vars="ym", var_name="item", value_name="value")

# 2) 기준 시계열 추출
bench = (
    uss_long.loc[uss_long["item"] == REF_ITEM, ["ym", "value"]]
            .rename(columns={"value": "bench_value"})
)

# 3) 모든 아이템에 기준값 머지
dfc = uss_long.merge(bench, on="ym", how="left")

# 4) '자기 자신' 라인과 '기준' 라인 두 벌로 쌓기
plot_df = pd.concat(
    [
        dfc.assign(series="self",  val=dfc["value"]),
        dfc.assign(series=f"benchmark: {REF_ITEM}", val=dfc["bench_value"])
    ],
    ignore_index=True
)

# 5) 그리기: 패싯은 item으로, 색은 series로
fig = px.line(
    plot_df,
    x="ym",
    y="val",
    color="series",
    facet_col="item",
    facet_col_wrap=6,
    facet_row_spacing=0.02,
    # 원하시면 색을 고정해 가독성을 높일 수 있습니다.
    color_discrete_map={
        "self": "#2962FF",                 # 각 아이템(자기 자신)
        f"benchmark: {REF_ITEM}": "#727272"  # 기준 라인(회색)
    }
)

# 패싯 라벨 정리 & 범례/레이아웃
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]),)
fig.update_traces(opacity=0.95)
# fig.update_layout(width=1800, height=1400, showlegend=True)
fig.update_layout(showlegend=True,)

# 필요하면 Y축 범위 고정(주의: 기준과 스케일이 다르면 왜곡될 수 있음)
# fig.update_yaxes(range=[0, 150])

fig.show()

## 4. 회귀 모델 학습
- 시계열 데이터 안에서 '한 달 뒤 총 무역량(value)을 맞추는 문제'로 self-supervised 학습
- 탐색된 모든 공행성쌍 (A,B)에 대해 월 t마다 학습 샘플 생성
- input X:
1) B_t (현재 총 무역량(value))
2) B_{t-1} (직전 달 총 무역량(value))
3) A_{t-lag} (lag 반영된 총 무역량(value))
4) max_corr, best_lag (관계 특성)
- target y:
1) B_{t+1} (다음 달 총 무역량(value))
- 이러한 모든 샘플을 합쳐 LinearRegression 회귀 모델을 학습

In [18]:
def build_training_data(pivot, pairs):
    """
    공행성쌍 + 시계열을 이용해 (X, y) 학습 데이터를 만드는 함수
    input X:
      - b_t, b_t_1, a_t_lag, max_corr, best_lag
    target y:
      - b_t_plus_1
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        n_eff = float(row.n_eff)
        ci_low = float(row.ci_low)
        ci_high = float(row.ci_high)
        ci_diff = float(row.ci_diff)


        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag, 1), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]
            month = ((t+1)% 12)+1

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "n_eff":n_eff,
                "ci_low":ci_low,
                "ci_high":ci_high,
                "ci_diff":ci_diff,
                "month":month,
                "target": b_t_plus_1,
                
            })

    df_train = pd.DataFrame(rows)
    return df_train

# df_train_model = build_training_data(df_minmax.set_index(df_minmax["ym"]).drop("ym",axis=1).T, pairs)
df_train_model = build_training_data(pivot, pairs)
print('생성된 학습 데이터의 shape :', df_train_model.shape)
df_train_model.head()

생성된 학습 데이터의 shape : (113457, 11)


,b_t,b_t_1,a_t_lag,max_corr,best_lag,n_eff,ci_low,ci_high,ci_diff,month,target
0,582317.0,539873.0,14276.0,-0.443984,5.0,5.477658,-0.938145,0.645756,1.583901,7,759980.0
1,759980.0,582317.0,52347.0,-0.443984,5.0,5.477658,-0.938145,0.645756,1.583901,8,216019.0
2,216019.0,759980.0,53549.0,-0.443984,5.0,5.477658,-0.938145,0.645756,1.583901,9,537693.0
3,537693.0,216019.0,0.0,-0.443984,5.0,5.477658,-0.938145,0.645756,1.583901,10,205326.0
4,205326.0,537693.0,26997.0,-0.443984,5.0,5.477658,-0.938145,0.645756,1.583901,11,169440.0


In [19]:
df_train_model.corr()

,b_t,b_t_1,a_t_lag,max_corr,best_lag,n_eff,ci_low,ci_high,ci_diff,month,target
b_t,1.000000,0.944131,-0.001016,-0.009462,-0.023466,-0.094327,-0.126546,0.069007,0.136784,-0.008407,0.945068
b_t_1,0.944131,1.000000,-0.001468,-0.007213,-0.025347,-0.096290,-0.128243,0.072058,0.139879,-0.004724,0.937614
a_t_lag,-0.001016,-0.001468,1.000000,0.007430,0.024305,-0.107588,-0.134965,0.091983,0.156784,0.009345,-0.002948
max_corr,-0.009462,-0.007213,0.007430,1.000000,-0.009839,-0.001177,0.495247,0.602322,-0.018120,0.000628,-0.011622
best_lag,-0.023466,-0.025347,0.024305,-0.009839,1.000000,-0.014487,-0.008990,0.000389,0.007041,0.019226,-0.023327
n_eff,-0.094327,-0.096290,-0.107588,-0.001177,-0.014487,1.000000,0.585633,-0.533299,-0.759853,0.000371,-0.093048
ci_low,-0.126546,-0.128243,-0.134965,0.495247,-0.008990,0.585633,1.000000,-0.082981,-0.806813,0.000893,-0.125381
ci_high,0.069007,0.072058,0.091983,0.602322,0.000389,-0.533299,-0.082981,1.000000,0.655720,-0.000078,0.066777
ci_diff,0.136784,0.139879,0.156784,-0.018120,0.007041,-0.759853,-0.806813,0.655720,1.000000,-0.000723,0.134579
month,-0.008407,-0.004724,0.009345,0.000628,0.019226,0.000371,0.000893,-0.000078,-0.000723,1.000000,-0.005729


In [64]:
def build_training_data_safe(pivot, pairs, window=24):
    months = pivot.columns.to_list()
    n_months = len(months)
    rows = []
    for row in pairs.itertuples(index=False):
        leader, follower, lag = row.leading_item_id, row.following_item_id, int(row.best_lag)
        if leader not in pivot.index or follower not in pivot.index: 
            continue
        a = pivot.loc[leader].values.astype(float)
        b = pivot.loc[follower].values.astype(float)

        # t 시점 예측에서: [t-window+1, t] 만 사용해 lag/corr 등의 요약을 계산
        for t in range(max(lag, 1, window-1), n_months-1):
            past_a = a[t-window+1:t+1]
            past_b = b[t-window+1:t+1]

            # 과거 구간에서의 상관(안정화/클리핑 권장)
            # lag 적용 후 과거-내 상관
            if lag < len(past_a):
                r = np.corrcoef(past_a[:-lag], past_b[lag:])[0,1]
            else:
                r = np.nan

            rows.append({
                "b_t": b[t],
                "b_t_1": b[t-1],
                "a_t_lag": a[t-lag],
                "corr_past": np.nan_to_num(r, nan=0.0, posinf=0.0, neginf=0.0),
                "best_lag": float(lag),       # pairs에서 가져오되, 전기간 산출치면 사용 자제/고정
                "target": b[t+1],
            })
    return pd.DataFrame(rows)
# df_train_model = build_training_data_safe(pivot, pairs)
# print('생성된 학습 데이터의 shape :', df_train_model.shape)
# df_train_model.head()

In [81]:
df_train_model.columns

Index(['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag', 'n_eff', 'ci_low',
       'ci_high', 'ci_diff', 'month', 'target'],
      dtype='object')

In [ ]:
pivot.T["DDEXPPXU"].iloc[2:]

ym
2022-03    38475.0
2022-04    23209.0
2022-05    37804.0
2022-06    27145.0
2022-07     1210.0
2022-08     5943.0
2022-09    16182.0
2022-10     7952.0
2022-11    21874.0
2022-12    23697.0
2023-01     7631.0
2023-02     9710.0
2023-03    21812.0
2023-04    20134.0
2023-05     4071.0
2023-06    10181.0
2023-07    44389.0
2023-08     1806.0
2023-09    11867.0
2023-10    17757.0
2023-11     1670.0
2023-12    17361.0
2024-01      973.0
2024-02    28079.0
2024-03    49166.0
2024-04     4721.0
2024-05    48382.0
2024-06    21403.0
2024-07     1668.0
2024-08    44847.0
2024-09    37743.0
2024-10    41553.0
2024-11     7570.0
2024-12    37825.0
2025-01    31225.0
2025-02     1041.0
2025-03     8390.0
2025-04    14627.0
2025-05    13967.0
2025-06    16467.0
2025-07    12017.0
Name: DDEXPPXU, dtype: float64

In [ ]:
test_df = pivot.T[["AANGBULD"]].iloc[:-2]
test_df["DDEXPPXU_laged"] = pivot.T["DDEXPPXU"].iloc[2:].values
test_df.head()

item_id,AANGBULD,DDEXPPXU_laged
ym,,
2022-01,14276.0,38475.0
2022-02,52347.0,23209.0
2022-03,53549.0,37804.0
2022-04,0.0,27145.0
2022-05,26997.0,1210.0


In [ ]:
unique_corr= df_train_model["max_corr"].unique()

In [20]:
# 회귀모델 학습
# feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag', 'n_eff', 'ci_low', 'ci_high','ci_diff','month']

train_X = df_train_model.drop("target",axis=1)
train_y = df_train_model["target"].values

In [21]:
reg = LinearRegression()
reg.fit(train_X, train_y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [22]:
model_cat = CatBoostRegressor(random_seed=42)
model_cat.fit(train_X, train_y)

Learning rate set to 0.086466
0:	learn: 12582570.8071369	total: 152ms	remaining: 2m 31s
1:	learn: 11635316.3669501	total: 159ms	remaining: 1m 19s
2:	learn: 10770372.7717482	total: 166ms	remaining: 55s
3:	learn: 9988645.5419741	total: 172ms	remaining: 42.9s
4:	learn: 9273489.0803880	total: 179ms	remaining: 35.7s
5:	learn: 8637582.3494659	total: 187ms	remaining: 30.9s
6:	learn: 8054409.5569320	total: 195ms	remaining: 27.6s
7:	learn: 7537158.5742688	total: 202ms	remaining: 25s
8:	learn: 7058677.0271242	total: 209ms	remaining: 23s
9:	learn: 6626834.4715155	total: 217ms	remaining: 21.5s
10:	learn: 6245505.0166208	total: 225ms	remaining: 20.2s
11:	learn: 5902935.9389041	total: 231ms	remaining: 19s
12:	learn: 5602255.6517691	total: 238ms	remaining: 18.1s
13:	learn: 5334354.5430902	total: 246ms	remaining: 17.3s
14:	learn: 5096060.2537281	total: 253ms	remaining: 16.6s
15:	learn: 4884675.3051626	total: 260ms	remaining: 16s
16:	learn: 4694902.9166205	total: 267ms	remaining: 15.5s
17:	learn: 45229

In [ ]:
model = lgb.LGBMRegressor()
model.fit(train_X, train_y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000676 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1792
[LightGBM] [Info] Number of data points in the train set: 88977, number of used features: 8
[LightGBM] [Info] Start training from score 4687838.283815


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


## 5. 회귀 모델 추론 및 제출(submission) 파일 생성
- 탐색된 공행성 쌍에 대해 후행 품목(following_item_id)에 대한 2025년 8월 총 무역량(value) 예측

In [23]:
def predict(pivot, pairs, reg):
    months = pivot.columns.to_list()
    n_months = len(months)

    # 가장 마지막 두 달 index (2025-7, 2025-6)
    t_last = n_months - 1
    t_prev = n_months - 2

    preds = []

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        n_eff = float(row.n_eff)
        ci_low = float(row.ci_low)
        ci_high = float(row.ci_high)
        ci_diff = float(row.ci_diff)
        month = int(8)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t_last - lag 가 0 이상인 경우만 예측
        if t_last - lag < 0:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        X_test = np.array([[b_t, b_t_1, a_t_lag, corr, float(lag),n_eff,ci_low,ci_high,ci_diff,month]])
        # X_test = np.array([[b_t, b_t_1, a_t_lag, corr, float(lag),month]])
        y_pred = reg.predict(X_test)[0]
        # (후처리 1) 음수 예측 → 0으로 변환
        # (후처리 2) 소수점 → 정수 변환 (무역량은 정수 단위)
        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    df_pred = pd.DataFrame(preds)
    return df_pred

In [ ]:
def _corr_in_past_window(a_series, b_series, t_last, window, lag):
    """
    t_last 시점까지의 과거 구간[t_last-window+1, t_last]만 사용하여
    lag을 적용한 (a→b) 상관을 계산합니다.
    """
    # 윈도우 시작/끝 인덱스
    start = max(0, t_last - window + 1)
    end = t_last  # inclusive

    past_a = a_series[start:end+1]
    past_b = b_series[start:end+1]
    # lag을 적용하려면 최소 길이가 lag+2 이상이어야 유효(상관 최소 2포인트)
    if len(past_a) <= lag + 1:
        return 0.0

    x = past_a[:-lag]   # a_{t-lag}
    y = past_b[lag:]    # b_t

    # 길이 확인
    n = min(len(x), len(y))
    if n < 2:
        return 0.0

    # 상관 계산(안정화)
    try:
        r = float(np.corrcoef(x[:n], y[:n])[0, 1])
        if not np.isfinite(r):
            r = 0.0
    except Exception:
        r = 0.0
    return r


def predict_safe(pivot, pairs, reg, window=24, clip_negative_to_zero=True, round_to_int=True):
    """
    안전한(누수 방지) 피처셋으로 예측.
    - 입력 피처: b_t, b_t_1, a_t_lag, corr_past, best_lag
    - corr_past: t_last 까지의 과거 window만으로 계산
    - best_lag: pairs에서 고정 사용(단, 전기간 산출치라면 학습과 동일 규칙 유지 필수)

    반환: follower(타겟)별 예측값 집계 DataFrame
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    # 예측 기준 시점(마지막 관측월의 다음 달을 예측한다고 가정)
    t_last = n_months - 1      # 마지막 관측월 인덱스
    t_prev = n_months - 2      # 그 이전 월

    preds = []

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # 필요한 값이 존재하는지(인덱스 범위) 체크
        if t_last - lag < 0 or t_prev < 0:
            continue

        b_t     = b_series[t_last]
        b_t_1   = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        # 과거 윈도우에서의 상관
        corr_past = _corr_in_past_window(a_series, b_series, t_last, window, lag)

        X_test = np.array([[b_t, b_t_1, a_t_lag, corr_past, float(lag)]])
        y_hat = float(reg.predict(X_test)[0])

        # 후처리(학습 시와 동일 규칙 유지)
        if clip_negative_to_zero:
            y_hat = max(0.0, y_hat)
        if round_to_int:
            y_hat = int(round(y_hat))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_hat,
            "b_t": b_t,
            "b_t_1": b_t_1,
            "a_t_lag": a_t_lag,
            "corr_past": corr_past,
            "best_lag": lag,
        })

    df_pred = pd.DataFrame(preds)

    # 같은 follower(타겟)에 여러 페어가 있을 수 있으니, 집계 방식 선택:
    # 1) 최대값(보수적), 2) 평균값(부드럽게), 3) 가중평균(|corr_past| 가중치) 등
    if not df_pred.empty:
        # 예: |corr_past| 가중 평균
        df_pred["abs_w"] = df_pred["corr_past"].abs().replace(0, 1e-6)
        agg = (df_pred
               .groupby("following_item_id", as_index=False)
               .apply(lambda g: pd.Series({
                   "value_pred_weighted": int(round(np.average(g["value"].astype(float), weights=g["abs_w"]))),
                   "value_pred_mean":     int(round(g["value"].mean())),
                   "value_pred_max":      int(round(g["value"].max())),
                   "num_pairs":           int(len(g))
               }))
              )
        # 원시 행도 함께 보려면 df_pred를 그대로 반환하거나, (agg, df_pred) 튜플 반환 가능
        return agg

    return df_pred  # 빈 경우 원시 반환

In [26]:
submission = predict(pivot, pairs, model_cat)
# submission = predict(pivot, pairs, reg)
submission.head()

2951it [00:04, 631.55it/s]


,leading_item_id,following_item_id,value
0,AANGBULD,APQGTRMF,95098
1,AANGBULD,BEZYMBBT,3652040
2,AANGBULD,DDEXPPXU,101273
3,AANGBULD,DEWLVASR,503710
4,AANGBULD,DNMPSKTB,4906209


In [27]:
submission.to_csv('./baseline_test_cat4.csv', index=False)

In [12]:
sub1 = pd.read_csv("baseline_submit8.csv")
sub2 = pd.read_csv("baseline_max_lag=6,min_nonzero=29,corr_threshold=0.3,cat.csv")